## Импорты и проч

In [1]:
# Импортируем необходимые библиотеки и функции
import os
import json
import numpy as np
import pandas as pd
import pysam
import tensorflow as tf
import time

from baskerville import seqnn, gene as bgene
from borzoi_helpers import process_sequence, predict_tracks  # предполагается, что эти функции доступны

# Отключаем лишние предупреждения TensorFlow
#tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


2025-03-12 18:54:18.151920: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-12 18:54:18.229355: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-12 18:54:18.229413: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-12 18:54:18.229548: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-12 18:54:18.251963: I tensorflow/core/platform/cpu_feature_g

In [2]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
print(tf.__version__)
!nvcc -V

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
2.14.0
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [3]:
%%bash

#Download model weights (data fold 3, 4 replicates)
for rep in f3c0,f0 f3c1,f1 f3c2,f2 f3c3,f3; do IFS=","; set -- $rep; 
  mkdir -p "saved_models/$1/train"
  local_model="saved_models/$1/train/model0_best.h5"
  if [ -f "$local_model" ]; then
    echo "$1 model already exists."
  else
    wget --progress=bar:force "https://storage.googleapis.com/seqnn-share/borzoi/$2/model0_best.h5" -O "$local_model"
  fi
done

#Download and uncompress annotation files
mkdir -p hg38/genes/gencode41
mkdir -p hg38/genes/polyadb

if [ -f hg38/genes/gencode41/gencode41_basic_nort.gtf ]; then
  echo "Gene annotation already exists."
else
  wget -O - https://storage.googleapis.com/seqnn-share/helper/gencode41_basic_nort.gtf.gz | gunzip -c > hg38/genes/gencode41/gencode41_basic_nort.gtf
fi

if [ -f hg38/genes/gencode41/gencode41_basic_nort_protein.gtf ]; then
  echo "Gene annotation (no read-through, protein-coding) already exists."
else
  wget -O - https://storage.googleapis.com/seqnn-share/helper/gencode41_basic_nort_protein.gtf.gz | gunzip -c > hg38/genes/gencode41/gencode41_basic_nort_protein.gtf
fi

if [ -f hg38/genes/gencode41/gencode41_basic_protein.gtf ]; then
  echo "Gene annotation (protein-coding) already exists."
else
  wget -O - https://storage.googleapis.com/seqnn-share/helper/gencode41_basic_protein.gtf.gz | gunzip -c > hg38/genes/gencode41/gencode41_basic_protein.gtf
fi

if [ -f hg38/genes/gencode41/gencode41_basic_tss2.bed ]; then
  echo "TSS annotation already exists."
else
  wget -O - https://storage.googleapis.com/seqnn-share/helper/gencode41_basic_tss2.bed.gz | gunzip -c > hg38/genes/gencode41/gencode41_basic_tss2.bed
fi

if [ -f hg38/genes/gencode41/gencode41_basic_protein_splice.csv.gz ]; then
  echo "Splice site annotation already exist."
else
  wget https://storage.googleapis.com/seqnn-share/helper/gencode41_basic_protein_splice.csv.gz -O hg38/genes/gencode41/gencode41_basic_protein_splice.csv.gz
fi

if [ -f hg38/genes/gencode41/gencode41_basic_protein_splice.gff ]; then
  echo "Splice site annotation already exist."
else
  wget -O - https://storage.googleapis.com/seqnn-share/helper/gencode41_basic_protein_splice.gff.gz | gunzip -c > hg38/genes/gencode41/gencode41_basic_protein_splice.gff
fi

if [ -f hg38/genes/polyadb/polyadb_human_v3.csv.gz ]; then
  echo "PolyA site annotation already exist."
else
  wget https://storage.googleapis.com/seqnn-share/helper/polyadb_human_v3.csv.gz -O hg38/genes/polyadb/polyadb_human_v3.csv.gz
fi

#Download and index hg38 genome
mkdir -p hg38/assembly/ucsc

if [ -f hg38/assembly/ucsc/hg38.fa ]; then
  echo "Human genome FASTA already exists."
else
  wget -O - http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz | gunzip -c > hg38/assembly/ucsc/hg38.fa
fi


f3c0 model already exists.
f3c1 model already exists.
f3c2 model already exists.
f3c3 model already exists.
Gene annotation already exists.
Gene annotation (no read-through, protein-coding) already exists.
lready exists.n (protein-coding) a
TSS annotation already exists.
Splice site annotation already exist.
Splice site annotation already exist.
PolyA site annotation already exist.
Human genome FASTA already exists.


## Препроцессинг

Код для понимания какой индекс отвечает за какую клеточную линию

In [3]:
# ---  Препроцессинг ---

targets_file = 'targets_human.txt'
col_names = [
    "identifier",
    "file_id",
    "file_path",
    "clip",
    "clip_soft",
    "scale",
    "sum_stat",
    "strand_pair",
    "description",
]

targets_df = pd.read_csv(targets_file, sep='\t', index_col=0, names=col_names)
display(targets_df.head(10))

# Эксперименты (судя по условию, ищем их в столбце "file_path")
experiment_ids = [
    "ENCSR892LBU",
    "ENCSR357BYU",
    "ENCSR763OMY",
    "ENCSR071DYD",
    "ENCSR045GTF",
]

def matches_exps(fp):
    return any(eid in fp for eid in experiment_ids)

filtered_df = targets_df[targets_df['file_path'].apply(matches_exps)].copy()
filtered_df['local_index'] = range(len(filtered_df))

# Добавляем столбец file_name, который будет извлечён из file_path
# Мы добавляем символы + или - в конец, если они присутствуют в file_id
filtered_df['file_name'] = filtered_df.apply(
    lambda row: f"{row['file_path'].split('/')[-3]}{row['file_id'][-1]}" if row['file_id'][-1] in ['+', '-'] else row['file_path'].split("/")[-3],
    axis=1
)

print("Эксперименты, которые подходят (в file_path содержат нужные ID):")
display(filtered_df)

# Собираем список глобальных индексов, потом приводим к int32, чтобы не было float
target_index = filtered_df.index.to_numpy(dtype='int32')  # <-- ВАЖНО: dtype='int32'
print("\nСписок глобальных индексов (int32) таргетов:", target_index)


,file_id,file_path,clip,clip_soft,scale,sum_stat,strand_pair,description
identifier,,,,,,,,
NaN,identifier,file,clip,clip_soft,scale,sum_stat,strand_pair,description
0.0,CNhs10608+,/home/drk/tillage/datasets/human/cage/fantom/C...,768,384,1.0,sum,1,CAGE:Clontech Human Universal Reference Total ...
1.0,CNhs10608-,/home/drk/tillage/datasets/human/cage/fantom/C...,768,384,1.0,sum,0,CAGE:Clontech Human Universal Reference Total ...
2.0,CNhs10610+,/home/drk/tillage/datasets/human/cage/fantom/C...,768,384,1.0,sum,3,CAGE:SABiosciences XpressRef Human Universal T...
3.0,CNhs10610-,/home/drk/tillage/datasets/human/cage/fantom/C...,768,384,1.0,sum,2,CAGE:SABiosciences XpressRef Human Universal T...
4.0,CNhs10612+,/home/drk/tillage/datasets/human/cage/fantom/C...,768,384,1.0,sum,5,CAGE:Universal RNA - Human Normal Tissues Bioc...
5.0,CNhs10612-,/home/drk/tillage/datasets/human/cage/fantom/C...,768,384,1.0,sum,4,CAGE:Universal RNA - Human Normal Tissues Bioc...
6.0,CNhs10615+,/home/drk/tillage/datasets/human/cage/fantom/C...,768,384,1.0,sum,7,"CAGE:adipose tissue, adult, pool1"
7.0,CNhs10615-,/home/drk/tillage/datasets/human/cage/fantom/C...,768,384,1.0,sum,6,"CAGE:adipose tissue, adult, pool1"


Эксперименты, которые подходят (в file_path содержат нужные ID):


,file_id,file_path,clip,clip_soft,scale,sum_stat,strand_pair,description,local_index,file_name
identifier,,,,,,,,,,
6396.0,ENCFF905NZB+,/home/drk/tillage/datasets/human/rna/encode/EN...,768,384,0.3,sum_sqrt,6397,RNA:lung tissue female adult (47 years),0,ENCSR045GTF+
6397.0,ENCFF905NZB-,/home/drk/tillage/datasets/human/rna/encode/EN...,768,384,0.3,sum_sqrt,6396,RNA:lung tissue female adult (47 years),1,ENCSR045GTF-
6423.0,ENCFF656OUX+,/home/drk/tillage/datasets/human/rna/encode/EN...,768,384,0.3,sum_sqrt,6424,RNA:pancreas tissue female child (16 years),2,ENCSR071DYD+
6424.0,ENCFF656OUX-,/home/drk/tillage/datasets/human/rna/encode/EN...,768,384,0.3,sum_sqrt,6423,RNA:pancreas tissue female child (16 years),3,ENCSR071DYD-
6730.0,ENCFF253SBE+,/home/drk/tillage/datasets/human/rna/encode/EN...,768,384,0.3,sum_sqrt,6731,RNA:left lobe of liver tissue male adult (45 y...,4,ENCSR357BYU+
6731.0,ENCFF253SBE-,/home/drk/tillage/datasets/human/rna/encode/EN...,768,384,0.3,sum_sqrt,6730,RNA:left lobe of liver tissue male adult (45 y...,5,ENCSR357BYU-
7240.0,ENCFF133LYJ+,/home/drk/tillage/datasets/human/rna/encode/EN...,768,384,0.3,sum_sqrt,7241,RNA:adrenal gland tissue female adult (41 years),6,ENCSR763OMY+
7241.0,ENCFF133LYJ-,/home/drk/tillage/datasets/human/rna/encode/EN...,768,384,0.3,sum_sqrt,7240,RNA:adrenal gland tissue female adult (41 years),7,ENCSR763OMY-
7396.0,ENCFF649RYB+,/home/drk/tillage/datasets/human/rna/encode/EN...,768,384,0.3,sum_sqrt,7397,RNA:kidney tissue female adult (47 years),8,ENCSR892LBU+



Список глобальных индексов (int32) таргетов: [6396 6397 6423 6424 6730 6731 7240 7241 7396 7397]


In [4]:
# --- Получения списка интервалов и их мерж ---

import pandas as pd
import gzip
from tqdm import tqdm

bed_file_path = '../data/sequences_human.bed.gz'
bed_data = pd.read_csv(
    bed_file_path,
    sep='\t',
    header=None,
    names=["chrom", "start", "end", "fold"],
    compression='gzip'
)

folds_to_process = ['fold3']  # Можно изменить
filtered_bed_data = bed_data[ bed_data['fold'].isin(folds_to_process) ]
grouped = filtered_bed_data.groupby("chrom")

print(len(filtered_bed_data),"строк до мерджа.")

def merge_intervals(intervals):
    intervals.sort(key=lambda x: x[0])
    merged = []
    for st, en in intervals:
        if not merged or st > merged[-1][1]:
            merged.append([st, en])
        else:
            merged[-1][1] = max(merged[-1][1], en)
    return merged

merged_intervals_list = []
unique_chroms = filtered_bed_data['chrom'].nunique()
for chrom, group in tqdm(filtered_bed_data.groupby("chrom"), desc="Мержим интервалы", total=unique_chroms):
    intervals = group[['start','end']].values.tolist()
    merged = merge_intervals(intervals)
    fold_val = group['fold'].iloc[0]
    for st, en in merged:
        merged_intervals_list.append({'chrom': chrom, 'start': st, 'end': en, 'fold': fold_val})

filtered_bed_data = pd.DataFrame(merged_intervals_list)
print("После мерджа:", len(filtered_bed_data))
display(filtered_bed_data.head())


6888 строк до мерджа.


Мержим интервалы: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 1029.98it/s]

После мерджа: 46


,chrom,start,end,fold
0,chr1,143479625,143823752,fold3
1,chr1,227321006,228550247,fold3
2,chr11,25440674,48748592,fold3
3,chr11,48896195,49191149,fold3
4,chr11,54525074,54820028,fold3


## Проверка данных

Убедимся что все записи влезут во входящее окно модели

In [6]:
# Находим максимальную длину последовательности
max_sequence_length = 0

for index, row in filtered_bed_data.iterrows():
    start = row['start']
    end = row['end']
    
    # Вычисляем длину текущей последовательности
    sequence_length = end - start
    
    # Обновляем максимальную длину, если текущая больше
    if sequence_length > max_sequence_length:
        max_sequence_length = sequence_length

print(f"Максимальная длина последовательности: {max_sequence_length} нуклеотидов")


Максимальная длина последовательности: 67957002 нуклеотидов


Проверка на пересечения

In [7]:
import pandas as pd
from tqdm import tqdm

overlap_found = False

# Получаем список уникальных хромосом
chromosomes = filtered_bed_data['chrom'].unique()

# Используем tqdm для отображения прогресса
for chrom in tqdm(chromosomes, desc="Checking for overlaps"):
    # Фильтруем по текущей хромосоме
    group = filtered_bed_data[filtered_bed_data['chrom'] == chrom]
    
    # Сортируем записи по старту
    sorted_group = group.sort_values('start')
    
    # Инициализируем предыдущую запись
    prev_row = None
    
    # Проходим по отсортированным записям
    for idx, row in sorted_group.iterrows():
        if prev_row is not None:
            # Проверяем пересечение
            if row['start'] < prev_row['end']:
                print(f"Пересечение на {chrom}: {prev_row[['start', 'end']].to_dict()} и {row[['start', 'end']].to_dict()}")
                overlap_found = True
        prev_row = row

if not overlap_found:
    print("Пересечений не обнаружено в filtered_bed_data.")


Checking for overlaps: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 1105.77it/s]

Пересечений не обнаружено в filtered_bed_data.


In [8]:
import pandas as pd
import gzip

# Загрузим данные из bed-файла
bed_file_path = '../data/sequences_human.bed.gz'
with gzip.open(bed_file_path, 'rt') as file:
    bed_data = pd.read_csv(file, sep='\t', header=None, 
                           names=["chrom", "start", "end", "fold"])

# Фильтруем данные по fold3 и fold4
#folds_to_process = ['fold3', 'fold4']
folds_to_process = ['fold3']
filtered_bed_data = bed_data[bed_data['fold'].isin(folds_to_process)]

# Выведем список уникальных хромосом
unique_chroms = filtered_bed_data['chrom'].unique()
print("Уникальные хромосомы в выборке:")
print(unique_chroms)
print(f"Количество уникальных хромосом: {len(unique_chroms)}")

# Для каждой хромосомы находим минимальный start и максимальный end
chrom_stats = filtered_bed_data.groupby("chrom").agg(min_start=('start', 'min'),
                                                      max_end=('end', 'max'))
print("\nМинимальный start и максимальный end по хромосомам:")
print(chrom_stats)

if len(unique_chroms) > 1:
    print("\nВыборка содержит более одной хромосомы.")
else:
    print("\nВыборка содержит только одну хромосому.")


Уникальные хромосомы в выборке:
['chr5' 'chr11' 'chr6' 'chr9' 'chr13' 'chr2' 'chr8' 'chr7' 'chr17' 'chrX'
 'chr18' 'chr12' 'chr15' 'chr19' 'chr1' 'chr16' 'chr20']
Количество уникальных хромосом: 17

Минимальный start и максимальный end по хромосомам:
       min_start    max_end
chrom                      
chr1   143479625  228550247
chr11   25440674   56541083
chr12   32992234   34368994
chr13   40653298  102135774
chr15   20168638   34675550
chr16   33491584   33884884
chr17   81799133   83225066
chr18   13485786   15206757
chr19    3108622    9156817
chr2    68747521  129505661
chr20   30186694   30383302
chr5     8770274  103148800
chr6    99603327  170690035
chr7       10000   74778699
chr8     8158857   55824055
chr9    80588528  138217638
chrX     3230043   58109050

Выборка содержит более одной хромосомы.


## Модель

In [5]:
# --- ЯЧЕЙКА 5: Инициализация моделей (n_reps) ---

params_file = 'params_pred.json'
with open(params_file) as f:
    params = json.load(f)
params_model = params['model']
params_train = params['train']

# ВАЖНО: n_reps > 1 => несколько реплик
n_reps = 4  # или 1, если хотим 1
rc = False  # мы не усредняем rc, т.к. +/– цепи разные каналы
# Папки: f3c0, f3c1, f3c2, f3c3

models = []
for rep_ix in range(n_reps):
    model_file = f"saved_models/f3c{rep_ix}/train/model0_best.h5"

    seqnn_model = seqnn.SeqNN(params_model)
    seqnn_model.restore(model_file, 0)

    # Используем build_slice(...) по target_index (из ЯЧЕЙКИ 2)
    seqnn_model.build_slice(target_index)

    # build_ensemble
    seqnn_model.build_ensemble(rc, [0])
    
    models.append(seqnn_model)

print(f"Загружено {n_reps} моделей, каждая со срезом по {len(target_index)} каналам.")
print("Параметры первой модели:")
print(" stride =", models[0].model_strides[0],
      "crop =", models[0].target_crops[0],
      "tlen =", models[0].target_lengths[0])


2025-03-12 18:54:39.379792: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 23390 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:21:01.0, compute capability: 7.0
2025-03-12 18:54:39.380519: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 30965 MB memory:  -> device: 1, name: Tesla V100-PCIE-32GB, pci bus id: 0000:21:02.0, compute capability: 7.0


Загружено 4 моделей, каждая со срезом по 10 каналам.
Параметры первой модели:
 stride = 32 crop = 16 tlen = 16352


In [7]:
def debug_model(model, seq_len=524288):
    """
    Отладочная функция для проверки работы модели.
    Выводит:
      - модельную сводку (summary)
      - ключевые параметры: stride, crop, target_length
      - форму входного (dummy) массива
      - форму выходных предсказаний (для прямой и реверс-комплементарной последовательностей)
    """
    print("=== Model Summary ===")
    model.model.summary()
    
    print("\n=== Model Parameters ===")
    print(f"stride         = {model.model_strides[0]}")
    print(f"crop           = {model.target_crops[0]}")
    print(f"target_length  = {model.target_lengths[0]}")
    
    # Создадим dummy one-hot последовательность
    # Каждая строка будет ровно одной единицей на 4 элементах.
    dummy_seq = np.zeros((seq_len, 4), dtype=np.float32)
    for i in range(seq_len):
        dummy_seq[i, np.random.randint(0, 4)] = 1.0
    print("\nDummy sequence shape:", dummy_seq.shape)
    
    # Предсказание для прямой цепи
    dummy_pred = predict_tracks([model], dummy_seq)
    print("\n=== Raw Prediction Output (прямая цепь) ===")
    print("dummy_pred.shape =", dummy_pred.shape)
    # Ожидается форма вроде: [1, L_out, X, C]. Попробуем взять replicate=0 и strand=0:
    try:
        y_plus = dummy_pred[0, :, 0, :]
        print("После индексирования y_plus.shape =", y_plus.shape)
    except Exception as e:
        print("Ошибка индексирования y_plus:", e)
    
    # Предсказание для реверс-комплементарного входа
    def reverse_complement_onehot(seq_onehot):
        rev = np.flip(seq_onehot, axis=0)
        revcomp = rev[:, [3,2,1,0]]
        return revcomp

    dummy_seq_rc = reverse_complement_onehot(dummy_seq)
    dummy_pred_rc = predict_tracks([model], dummy_seq_rc)
    print("\n=== Raw Prediction Output (реверс-компл.) ===")
    print("dummy_pred_rc.shape =", dummy_pred_rc.shape)
    try:
        y_minus = dummy_pred_rc[0, :, 0, :]
        print("После индексирования y_minus.shape =", y_minus.shape)
    except Exception as e:
        print("Ошибка индексирования y_minus:", e)
    
    # Можно вывести также максимальные и минимальные значения предсказания
    print("\n=== Примеры статистики выходного тензора ===")
    print("Прямая цепь: min =", np.min(y_plus), "max =", np.max(y_plus))
    print("Реверс цепь: min =", np.min(y_minus), "max =", np.max(y_minus))


# Запускаем отладку модели:
debug_model(seqnn_model)


=== Model Summary ===
Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequence (InputLayer)       [(None, 524288, 4)]       0         
                                                                 
 model_13 (Functional)       (None, 16352, 7611)       185917723 
                                                                 
 tf.compat.v1.gather_3 (TFO  (None, 16352, 10)         0         
 pLambda)                                                        
                                                                 
Total params: 185917723 (709.22 MB)
Trainable params: 185892699 (709.12 MB)
Non-trainable params: 25024 (97.75 KB)
_________________________________________________________________

=== Model Parameters ===
stride         = 32
crop           = 16
target_length  = 16352

Dummy sequence shape: (524288, 4)


2025-02-22 03:03:47.550035: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902



=== Raw Prediction Output (прямая цепь) ===
dummy_pred.shape = (1, 1, 16352, 10)
После индексирования y_plus.shape = (1, 10)

=== Raw Prediction Output (реверс-компл.) ===
dummy_pred_rc.shape = (1, 1, 16352, 10)
После индексирования y_minus.shape = (1, 10)

=== Примеры статистики выходного тензора ===
Прямая цепь: min = 0.005516 max = 0.02629
Реверс цепь: min = 0.012474 max = 0.05438


In [6]:
# --- ЯЧЕЙКА 6: Загрузка FASTA, chr_sizes ---

fasta_path = "hg38/assembly/ucsc/hg38.fa"
fasta_index = pysam.Fastafile(fasta_path)
chr_sizes = {c: fasta_index.get_reference_length(c) for c in fasta_index.references}
print("Пример chr_sizes:", list(chr_sizes.items())[:3])


Пример chr_sizes: [('chr1', 248956422), ('chr10', 133797422), ('chr11', 135086622)]


## Инференс

In [7]:
# --- Формирование окон ---

center_len = 196_608
context_len = 163_840

all_windows = []
for _, row in filtered_bed_data.iterrows():
    chrom = row['chrom']
    st_merged = row['start']
    en_merged = row['end']
    curr_start = st_merged
    while curr_start < en_merged:
        curr_end = curr_start + center_len
        if curr_end > en_merged:
            curr_end = en_merged
        all_windows.append((chrom, curr_start, curr_end))
        curr_start = curr_end

print("Всего сформировано", len(all_windows), "окон.")


Всего сформировано 1785 окон.


In [9]:
# --- обратная трансформация ---

def inverse_transform(y_array, clip, clip_soft, scale, sum_stat):
    """
    Упрощённая логика «обратной» трансформации:
    1) Делим на scale (если scale != 1)
    2) Если sum_stat='sum_sqrt', снимаем clip_soft и sqrt
       (как в примере: y->(y-clip_soft+1)**2 + clip_soft -1, потом (y+1)**2 -1)
       иначе ничего не делаем.
    """
    # Преобразуем входные параметры к float,
    # чтобы избежать TypeError при делении (например, если они считались строками).
    clip = float(clip) if clip is not None else 0.0
    clip_soft = float(clip_soft) if clip_soft is not None else 0.0
    scale = float(scale) if scale is not None else 1.0

    # Переводим sum_stat в нижний регистр на всякий случай
    if isinstance(sum_stat, str):
        sum_stat = sum_stat.lower()
    else:
        sum_stat = ""

    # Копируем массив в float32
    y = np.array(y_array, dtype=np.float32)

    # 1) Делим на scale
    if scale != 1.0:
        y /= scale

    # 2) Если sum_stat='sum_sqrt', снимаем clip_soft + sqrt
    if sum_stat == 'sum_sqrt':
        if clip_soft > 0:
            # (y - clip_soft +1)**2 + clip_soft -1
            y_unclip = (y - clip_soft + 1.0)**2 + clip_soft - 1.0
            mask = (y > clip_soft)
            y[mask] = y_unclip[mask]
        # Далее снимаем sqrt+1 => (y+1)**2 -1 (упрощённая логика)
        y = (y + 1.0)**2 - 1.0

    return y


In [10]:
# --- Логика обработки входных данных для отрицательных цепей ---

def reverse_complement_onehot(seq_onehot):
    """
    Преобразует (L,4) → (L,4) реверс-комплемент с реверсированием.
    """
    rev = np.flip(seq_onehot, axis=0)  # Реверсируем последовательность
    revcomp = rev[:, [3, 2, 1, 0]]    # Меняем на комплементарные основания
    return revcomp

In [13]:
# --- Инференс с обратной трансформацией для + и - цепей (2 варианта: forward и reverse) ---
folds_str = "_".join(folds_to_process) 

def run_inference(all_windows, models):
    """
    Для каждого окна (chrom, cstart, cend) делаем 2 варианта:
     - forward: обычный вход
     - reverse: реверс-комплемент вход, с последующим обратным реверсированием выходов
    и записываем результаты (после обратной трансформации) в 2 отдельных файла на канал:
     borzoi_rnaseq_{folds_str}_{file_id}_fw.bedGraph
     borzoi_rnaseq_{folds_str}_{file_id}_rc.bedGraph

    При записи добавляется ручной сдвиг на 512 баз ( pos_start += 512 ).
    """
    start_time = time.time()
    print(f"Старт инференса (2 варианта) по {len(all_windows)} окнам.")

    stride = models[0].model_strides[0]
    crop   = models[0].target_crops[0]
    SHIFT  = 512  # ручной сдвиг координат

    def write_bedgraph(chrom, cstart, cend, values, file_name, suffix_label):
        """
        Записываем массив values в файл 
          borzoi_rnaseq_{folds_str}_{file_name}_{suffix_label}.bedGraph,
        учитывая смещение SHIFT.
        """
        bed_path = os.path.join(output_dir, f"borzoi_rnaseq_{folds_str}_{file_name}_{suffix_label}.bedGraph")
        for i, val in enumerate(values):
            pos_start = cstart + i * stride + SHIFT
            pos_end   = pos_start + stride
            if pos_end > cend + SHIFT:
                pos_end = cend + SHIFT
            if pos_end <= pos_start:
                break
            with open(bed_path, "a") as f_bg:
                f_bg.write(f"{chrom}\t{pos_start}\t{pos_end}\t{float(val)}\n")

    for (chrom, cstart, cend) in tqdm(all_windows, desc="Inference windows"):
        # 1) Формируем входные данные
        input_start = max(0, cstart - context_len)
        input_end   = min(chr_sizes[chrom], cend + context_len)

        seq_1hot = process_sequence(fasta_index, chrom, input_start, input_end)
        if seq_1hot is None or seq_1hot.shape[0] == 0:
            continue

        # 2) Предсказание для forward входа
        y_pred_fw = predict_tracks(models, seq_1hot)[0, 0, :, :]  # [L_out, C]

        # 3) Предсказание для reverse входа: сначала преобразуем вход,
        # затем получаем предсказание и после этого переворачиваем выход по оси 0
        seq_rc = reverse_complement_onehot(seq_1hot)
        y_pred_rc = predict_tracks(models, seq_rc)[0, 0, :, :]  # [L_out, C]
        y_pred_rc = np.flip(y_pred_rc, axis=0)

        L_out = y_pred_fw.shape[0]

        # 4) Определяем, какой кусок из предсказания соответствует [cstart, cend]
        offset_nt = stride * crop
        out_start_nt = offset_nt
        out_end_nt = offset_nt + stride * (L_out - 2 * crop)

        center_start_in_input = cstart - input_start
        center_end_in_input = cend - input_start

        slice_start_nt = max(center_start_in_input, out_start_nt)
        slice_end_nt = min(center_end_in_input, out_end_nt)
        if slice_end_nt <= slice_start_nt:
            continue

        out_slice_start = int((slice_start_nt - out_start_nt) // stride)
        out_slice_end = int(np.ceil((slice_end_nt - out_start_nt) / stride))

        if out_slice_start < 0:
            out_slice_start = 0
        if out_slice_end > (L_out - 2 * crop):
            out_slice_end = L_out - 2 * crop
        if out_slice_end <= out_slice_start:
            continue

        out_slice_start_full = crop + out_slice_start
        out_slice_end_full = crop + out_slice_end

        # 5) Выделяем нужные предсказания для forward и reverse вариантов
        pred_fw_slice = y_pred_fw[out_slice_start_full:out_slice_end_full, :]
        pred_rc_slice = y_pred_rc[out_slice_start_full:out_slice_end_full, :]

        length_slice = pred_fw_slice.shape[0]
        if length_slice <= 0:
            continue

        # 6) Для каждого канала делаем обратную трансформацию и записываем в соответствующие bedGraph файлы
        for channel_ix in range(pred_fw_slice.shape[1]):
            global_id = target_index[channel_ix]
            row = targets_df.loc[global_id]
            clip_val = row['clip']
            clip_soft_val = row['clip_soft']
            scale_val = row['scale']
            sum_stat_val = row['sum_stat']
            file_path = row['file_path']

            # Формируем правильное название файла из file_path
            experiment_name = file_path.split("/")[-3]
            if row['file_id'][-1] in ['+', '-']:
                file_name = f"{experiment_name}{row['file_id'][-1]}"
            else:
                file_name = experiment_name

            vals_fw = inverse_transform(pred_fw_slice[:, channel_ix], clip_val, clip_soft_val, scale_val, sum_stat_val)
            vals_rc = inverse_transform(pred_rc_slice[:, channel_ix], clip_val, clip_soft_val, scale_val, sum_stat_val)

            write_bedgraph(chrom, cstart, cend, vals_fw, file_name, "fw")
            write_bedgraph(chrom, cstart, cend, vals_rc, file_name, "rc")

    elapsed = time.time() - start_time
    print(f"Инференс завершён за {elapsed:.2f} секунд.")


In [ ]:
print("Start inference...")
run_inference(all_windows, models)
print("Done. BedGraph files have been saved.")

Start inference...
Старт инференса (2 варианта) по 1785 окнам.


Inference windows:   1%|▋                                                                                                                            | 9/1785 [01:18<4:12:24,  8.53s/it]

## Пост процесс

In [41]:
# ---  Пост-процессинг записанных bedGraph файлов (сдвиг координат) ---
# Не обязательная вещь, просто если предикты не попадают в гены можно легко дофиксить бегграфы этой ячейкой

import os

def shift_bedgraph_files(output_dir, shift=512):
    """
    Пост-процессинг: откроет все bedGraph файлы в output_dir и сдвигет их координаты на `shift` баз.
    """
    for file_name in os.listdir(output_dir):
        if file_name.endswith(".bedGraph"):
            file_path = os.path.join(output_dir, file_name)
            
            with open(file_path, 'r') as f:
                lines = f.readlines()

            with open(file_path, 'w') as f:
                for line in lines:
                    if line.startswith("track"):
                        f.write(line)  # Заголовок сохраняем без изменений
                    else:
                        # Разбираем строку bedGraph
                        chrom, pos_start, pos_end, val = line.strip().split("\t")
                        pos_start = int(pos_start) + shift
                        pos_end = int(pos_end) + shift
                        # Записываем сдвинутые координаты
                        f.write(f"{chrom}\t{pos_start}\t{pos_end}\t{val}\n")

    print(f"Сдвиг координат на {shift} баз завершен.")

# Запускаем сдвиг
shift_bedgraph_files("predicted_expression_by_chromosomes/")


Сдвиг координат на 512 баз завершен.


In [24]:
#НЕ ИСПОЛЬЗОВАТЬ!

import os
import shutil

# Папка с исходными файлами
input_dir = "predicted_expression_by_chromosomes/"
output_dir = "predicted_expression_by_chromosomes_renamed/"
os.makedirs(output_dir, exist_ok=True)

# Проходим по всем файлам в папке
for file_name in os.listdir(input_dir):
    if file_name.endswith(".bedGraph"):
        # Извлекаем название эксперимента из имени файла
        # Например, "borzoi_rnaseq_fold3_ENCFF133LYJ-_fw.bedGraph" -> "ENCFF133LYJ"
        experiment_name = file_name.split("_")[3]
        
        # Находим строку в targets_df, где file_id содержит experiment_name
        matching_row = targets_df[targets_df['file_id'].apply(lambda x: experiment_name in x)]
        
        if not matching_row.empty:
            # Получаем правильный file_path из строки
            file_path = matching_row['file_path'].values[0]
            
            # Строим новое имя файла, используя часть file_path
            # Например, из file_path '/home/drk/tillage/datasets/human/rna/encode/ENCSR045GTF/summary/coverage+.w5'
            # мы получим 'ENCSR045GTF' (часть пути)
            experiment_folder = file_path.split("/")[-3]
            
            # Строим новое имя файла, добавляя суффиксы fw или rc
            suffix = "_fw" if "_fw" in file_name else "_rc"
            new_file_name = f"borzoi_rnaseq_{folds_str}_{experiment_folder}{suffix}.bedGraph"
            
            # Полный путь к исходному и целевому файлу
            original_file_path = os.path.join(input_dir, file_name)
            new_file_path = os.path.join(output_dir, new_file_name)
            
            # Проверяем, существует ли файл и переименовываем
            if os.path.exists(original_file_path):
                shutil.move(original_file_path, new_file_path)
                print(f"Файл переименован: {file_name} -> {new_file_name}")
            else:
                print(f"Файл не найден: {file_name}")
        else:
            print(f"Эксперимент {experiment_name} не найден в file_id.")

print("Переименование файлов завершено.")


Файл переименован: borzoi_rnaseq_fold3_ENCFF133LYJ+_fw.bedGraph -> borzoi_rnaseq_fold3_ENCSR763OMY_fw.bedGraph
Файл переименован: borzoi_rnaseq_fold3_ENCFF253SBE-_rc.bedGraph -> borzoi_rnaseq_fold3_ENCSR357BYU_rc.bedGraph
Файл переименован: borzoi_rnaseq_fold3_ENCFF905NZB+_rc.bedGraph -> borzoi_rnaseq_fold3_ENCSR045GTF_rc.bedGraph
Файл переименован: borzoi_rnaseq_fold3_ENCFF649RYB+_fw.bedGraph -> borzoi_rnaseq_fold3_ENCSR892LBU_fw.bedGraph
Файл переименован: borzoi_rnaseq_fold3_ENCFF253SBE-_fw.bedGraph -> borzoi_rnaseq_fold3_ENCSR357BYU_fw.bedGraph
Файл переименован: borzoi_rnaseq_fold3_ENCFF656OUX-_rc.bedGraph -> borzoi_rnaseq_fold3_ENCSR071DYD_rc.bedGraph
Файл переименован: borzoi_rnaseq_fold3_ENCFF905NZB-_fw.bedGraph -> borzoi_rnaseq_fold3_ENCSR045GTF_fw.bedGraph
Файл переименован: borzoi_rnaseq_fold3_ENCFF905NZB+_fw.bedGraph -> borzoi_rnaseq_fold3_ENCSR045GTF_fw.bedGraph
Файл переименован: borzoi_rnaseq_fold3_ENCFF905NZB-_rc.bedGraph -> borzoi_rnaseq_fold3_ENCSR045GTF_rc.bedGraph
Ф